In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import re

# Load model and tokenizer
base_model = "mistralai/Mistral-7B-v0.1"
lora_path = "mistral-lora-attempt-new-10"

tokenizer = AutoTokenizer.from_pretrained(lora_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

model = PeftModel.from_pretrained(model, lora_path)
model.eval()

# Instruction for model
instruction = (
    "You are a knowledgeable and helpful health insurance expert. "
    "Provide general, easy-to-understand guidance about health insurance that applies broadly to anyone. "
    "Respond independently without referencing specific individuals, organizations, locations, country names, policies, procedures, or steps. "
    "Avoid using bullet points, numbered steps, costs, currency symbols (such as Rs, $, etc.), claim forms, or hyperlinks. "
    "Do not mention or suggest calling, submitting, registering, or logging into any system. "
    "Use clear, natural language and ensure your response fits within a moderate token limit. "
    "Keep answers friendly, neutral, and general—like universal advice that applies to everyone."
)

# Function to build prompt
def build_prompt(instruction, question):
    return (
        f"### Instruction:\n{instruction}\n\n"
        f"### Input:\n{question}\n\n"
        f"### Response:\n"
    )


# Function to truncate answer to last complete sentence
def truncate_to_complete_sentence(text):
    match = re.search(r'^(.*?[\.!?])[^\.!?]*$', text.strip(), re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()


# Questions List
questions = [
    "Why do we need health insurance?",
    "How do I get health insurance?",
    "What is health insurance?",
    "Why should I buy health insurance?",
    "How will health insurance pay for my emergency medical expenses?",
    "What is the eligible age to buy health insurance?",
    "Will I be allowed to cover my family under my health insurance?",
    "Will I be allowed to buy more than one health insurance plan?",
    "What are the types of health insurance plans I can buy?",
    "Can I pay my health insurance premium in installments?",
    "What are the common coverage benefits under my health insurance?",
    "What are the benefits of buying a critical illness insurance policy?",
    "What if I already have a health insurance policy, but I just want to increase my sum insured?",
    "I have been recently diagnosed with a medical condition. Will I be allowed health insurance?",
    "Will I gain more benefits by buying health insurance at an early age?",
    "What are the things I should consider before buying my health insurance policy?",
    "What is a health savings account, and what can I use those funds for?",
    "If my insurance company denies coverage for a service, is there anything I can do?",
    "What Does the Policy Cover?",
    "What is the Premium, are there any Copayments or Out-of-pocket costs?",
    "Are There Any Hospital Restrictions that I can’t visit?",
    "What is the Cover Limit?",
    "Are Pre-existing Conditions Covered?",
    "What is the Process for Filing Claims?",
    "Are There Any Additional Benefits, Discounts or Value Adds Available?",
    "How Does the Health Policy Handle Emergency and Urgent Care?",
    "What is the Renewal Process?",
    "Are There Any Exclusions or Limitations I Should Be Aware Of?",
    "If I’m having problems with my insurance, where can I file a complaint?",
    "What does the policy cover?",
    "How much does the plan cost?",
    "What the policy does not cover?",
    "How is the policy premium determined?",
    "How much money will you have to pay for availing medical care?",
    "What is the process for filing a claim?",
    "What is Health Insurance?",
    "Types of Health Insurance",
    "Benefits of Health Insurance",
    "How Does Health Insurance Work?",
    "Need for Health Insurance",
    "Why Should I Buy a Health Insurance Policy?",
    "What are the various types of health insurance?",
    "How to apply for Health Insurance?",
    "Health insurance features and benefits",
    "What things are covered under the hospitalisation benefit in health insurance?",
    "Can we add family members to existing health insurance?",
    "How do I make a claim?",
    "How Does Private Health Insurance Work?",
    "What Does Private Health Insurance Cover?",
    "Will My Health Insurance Plan Save Me Money If I’m Healthy?",
    "Will My Health Insurance Plan Be Affordable If I’m Ill?",
    "What Will My Potential Health Insurance Plan’s Prescription Drug Coverage Be Like?",
    "What Is a Pre-Existing Condition in Health Insurance?",
    "How long does it take to claim a life insurance policy?",
    "Can I claim back critical illness insurance?",
    "How long does it take MetLife to process a life insurance claim?",
    "How long does life insurance claim take?",
    "How to claim life insurance money?",
    "Can you claim disability insurance premiums on taxes?",
    "Can a life insurance claim be denied?"
]



# Generate and print answers 
for idx, question in enumerate(questions, 1):
    prompt = build_prompt(instruction, question)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            min_new_tokens=250,
            max_new_tokens=500,
            temperature=0.7,              
            top_p=0.95,                  
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True                # ← key change!
        )
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    raw_answer = decoded.split("### Response:\n")[-1].strip()
    answer = truncate_to_complete_sentence(raw_answer)

    print(f"\nQ{idx}: {question}\nA{idx}:\n{answer}\n{'-'*100}")


SyntaxError: incomplete input (1093646822.py, line 87)